# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazacorte,28.6290,-17.9293,19.54,80,100,2.57,ES,2023-05-25
1,1,ushuaia,-54.8000,-68.3000,0.81,80,20,3.60,AR,2023-05-25
2,2,iqaluit,63.7506,-68.5145,1.85,74,75,4.47,CA,2023-05-25
3,3,haiku-pauwela,20.9219,-156.3051,26.20,69,75,8.23,US,2023-05-25
4,4,vorgashor,67.5833,63.9500,13.26,76,96,4.19,RU,2023-05-25


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', c='Humidity', cmap='viridis', size='Humidity', alpha=0.6, hover_cols=['City', 'Country'], geo=True, tiles='OSM', frame_width=600, frame_height=400)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 25  # Maximum temperature in Celsius
humidity = 75  # Humidity percentage
cloudiness = 20  # Cloudiness percentage
wind_speed = 10  # Wind speed in meters per second

# Drop any rows with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) &
    (city_data_df['Humidity'] <= humidity) &
    (city_data_df['Cloudiness'] <= cloudiness) &
    (city_data_df['Wind Speed'] <= wind_speed)
].dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,richards bay,-28.7830,32.0377,23.13,63,2,7.42,ZA,2023-05-25
14,14,lubango,-14.9172,13.4925,15.36,49,0,1.23,AO,2023-05-25
15,15,kununurra,-15.7667,128.7333,17.99,72,4,1.03,AU,2023-05-25
32,32,bethel,41.3712,-73.4140,18.31,47,0,4.12,US,2023-05-25
39,39,broome,42.2506,-75.8330,15.81,49,0,5.14,US,2023-05-25
...,...,...,...,...,...,...,...,...,...,...
558,558,cinco saltos,-38.8222,-68.0629,15.85,25,0,7.20,AR,2023-05-25
561,561,darwin,-12.4611,130.8418,22.16,66,0,3.09,AU,2023-05-25
562,562,ronne,55.1009,14.7066,10.95,71,0,6.17,DK,2023-05-25
564,564,shubarshi,48.5872,57.1869,18.08,35,1,2.92,KZ,2023-05-25


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,richards bay,ZA,-28.7830,32.0377,63,
14,lubango,AO,-14.9172,13.4925,49,
15,kununurra,AU,-15.7667,128.7333,72,
32,bethel,US,41.3712,-73.4140,47,
39,broome,US,42.2506,-75.8330,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = { "apiKey": geoapify_key,
    "type": "accommodation.hotel",
    "limit": 1,  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon =row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = response
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
richards bay - nearest hotel: Protea Hotel
lubango - nearest hotel: Novo Hotel
kununurra - nearest hotel: Kununurra Country Club Resort
bethel - nearest hotel: Hampton Inn Danbury
broome - nearest hotel: No hotel found
egilsstadir - nearest hotel: Valaskjálf
qamdo - nearest hotel: 银杏商务宾馆
kone - nearest hotel: Pacifik Appartels
jamestown - nearest hotel: DoubleTree Jamestown
torbat-e jam - nearest hotel: No hotel found
el sauzal de rodriguez - nearest hotel: Hotel Sausalito
badger - nearest hotel: Hotel North Pole
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
zhezqazghan - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
sao vicente - nearest hotel: Pousada Vitória
nizhnesortymskiy - nearest hotel: Нижнесортымский
girua - nearest hotel: Hotel do Santo
happy valley-goose bay - nearest hotel: Hotel North Two
namibe - nearest hotel: Hotel Chik Chik Namibe
changji - nearest hotel: No hotel found
cambridge - nearest hotel: Travelo

,City,Country,Lat,Lng,Humidity,Hotel Name
8,richards bay,ZA,-28.7830,32.0377,63,Protea Hotel
14,lubango,AO,-14.9172,13.4925,49,Novo Hotel
15,kununurra,AU,-15.7667,128.7333,72,Kununurra Country Club Resort
32,bethel,US,41.3712,-73.4140,47,Hampton Inn Danbury
39,broome,US,42.2506,-75.8330,49,No hotel found
...,...,...,...,...,...,...
558,cinco saltos,AR,-38.8222,-68.0629,25,Hotel Global
561,darwin,AU,-12.4611,130.8418,66,Mantra Pantanas Darwin
562,ronne,DK,55.1009,14.7066,71,Det gamle posthus
564,shubarshi,KZ,48.5872,57.1869,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Country', 'Hotel Name'], hover=True, alpha=0.7, cmap='coolwarm', size='Humidity', title='City Map',frame_width=600, frame_height=400, geo= True, tiles='OSM')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)